In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import spacy
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jameelamer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/Users/jameelamer/MasterProject/Summarization/summrization_webapp/flaskProject2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel

In [4]:
# Load SBERT model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
# Load English tokenizer, POS tagger, parser, NER from spaCy
import spacy.cli

try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    import spacy.cli
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")
def normalize_whitespace(text):
    """Remove extra whitespace, newlines, and tabs."""
    return re.sub(r'\s+', ' ', text).strip()

def remove_noise_and_references(text):
    """
    Remove references like [1], (Smith et al., 2020), and figure/table mentions.
    You can extend the patterns as needed.
    """
    # Remove square bracket citations like [1], [12]
    text = re.sub(r'\[\d+\]', '', text)
    
    # Remove in-text references like (Smith et al., 2020)
    text = re.sub(r'\(([^)]*et al\.,?\s?\d{4})\)', '', text)
    
    # Remove "Fig. 1", "Table 2", etc.
    text = re.sub(r'(Fig\.?|Figure|Table)\s?\d+[a-zA-Z]?', '', text, flags=re.IGNORECASE)
    
    # Remove licensing and copyright boilerplate
    text = re.sub(r'©.*?(\.|\n)', '', text)
    text = re.sub(r'This article is licensed.*?(\.|\n)', '', text, flags=re.IGNORECASE)
    
    return text

def sentence_segmentation(text):
    """Segment text into individual sentences using spaCy."""
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

def preprocess_text(raw_text):
    """Complete preprocessing pipeline."""
    step1 = normalize_whitespace(raw_text)
    step2 = remove_noise_and_references(step1)
    sentences = sentence_segmentation(step2)
    return sentences
    #return ' '.join(str(sentence) for sentence in sentences)

In [6]:
# def preprocess_text(text):
#     sentences = text.split(". ")  # Simple sentence splitting
#     return [sent.strip() for sent in sentences if len(sent.strip()) > 10]

def get_bert_embeddings(sentences):
    return bert_model.encode(sentences, convert_to_tensor=True)

def apply_lda(sentences, num_topics=5):
    # Tokenization
    tokenized_sentences = [sent.lower().split() for sent in sentences]
    
    # LDA Preparation
    dictionary = Dictionary(tokenized_sentences)
    corpus = [dictionary.doc2bow(text) for text in tokenized_sentences]
    
    # Train LDA Model
    lda = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
    
    # Get sentence-topic probabilities
    topic_probs = []
    for sent in corpus:
        topic_dist = lda.get_document_topics(sent, minimum_probability=0)
        topic_probs.append([prob for _, prob in topic_dist])
    
    return np.array(topic_probs)

In [7]:
def generate_summary(text, summary_length=3):
    sentences = preprocess_text(text)
    embeddings = get_bert_embeddings(sentences)
    
    # Compute document embedding
    doc_embedding = np.mean(embeddings.cpu().numpy(), axis=0).reshape(1, -1)
    
    # Apply LDA
    topic_probs = apply_lda(sentences)
    
    # Sentence Ranking based on relevance
    similarities = cosine_similarity(embeddings.cpu().numpy(), doc_embedding).flatten()
    
    # Ensure topic diversity
    selected_sentences = []
    selected_indices = set()
    topic_order = np.argsort(-topic_probs.sum(axis=0))  # Prioritize top topics
    
    for topic in topic_order:
        topic_sent_indices = np.argsort(-topic_probs[:, topic])  # Sentences sorted by topic importance
        for idx in topic_sent_indices:
            if idx not in selected_indices:
                selected_sentences.append((sentences[idx], similarities[idx]))
                selected_indices.add(idx)
            if len(selected_sentences) >= summary_length:
                break
        if len(selected_sentences) >= summary_length:
            break
    
    # Sort selected sentences by relevance
    selected_sentences.sort(key=lambda x: -x[1])
    
    return " ".join([sent[0] for sent in selected_sentences])


In [8]:
document_text = """Y
القوات الأوكرانية تبدأ الانسحاب من القرم,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة القرم.,"وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينوف، قد أمر بسحب جميع القوات الأوكرانية من القرم. وسيطرت قوات روسية صباح الاثنين على قاعدة بحرية أوكرانية في فيودوسيا، في ثالث هجوم من نوعه خلال 48 ساعة، وذلك بحسب تصريحات مسؤولين أوكرانيين لبي بي سي . وقال المتحدث باسم وزارة الدفاع الأوكرانية فلاديسلاف سيليزنيوف إن القوات الروسية هاجمت القاعدة وألقت القبض على الجنود الأوكرانيين في قاعدة فيودوسيا وقيدت أيادي ضباطهم. ومن المتوقع أن تسيطر الأزمة الأوكرانية على قمة مجموعة الدول الصناعية السبع في لاهاي. مواضيع قد تهمك نهاية وأكد الرئيس الأمريكي باراك أوباما خلال لقاء مع نظيره الصيني شى جين بينغ على أن ""واشنطن وبكين يمكنهما، بالعمل سويا، تعزيز القانون الدولي واحترام سيادة الدول"". وتسيطر قوات روسية حاليا على معظم القواعد العسكرية الأوكرانية في القرم التي أعلنت موسكو ضمها للاتحاد الروسي بعد استفتاء أجرته السلطات المحلية هناك. قلق بالغ وقال مارك لوين، مراسل بي بي سي في القرم، إن القوات الروسية تسيطر بشكل كامل على القاعدة، ونقلت الجنود الأوكرانيين بعيدا إلى مكان مجهول. وتعد قاعدة فيودوسيا واحدة من آخر القواعد العسكرية التي بقيت تحت سيطرة كييف، لكن قوات روسية ظلت تحاصرها لبعض الوقت، حسبما أفاد مراسلنا. واقتحمت القوات الروسية قاعدتين أخريين وسيطرت عليهما يوم الجمعة. وكان مسؤولون عسكريون روس أعلنوا في وقت سابق أن العلم الروسي أصبح يرفرف على 189 وحدة ومنشأة عسكرية أوكرانية في القرم. وقال ديفيد ستيرن مراسل بي بي سي في كييف إن الأوكرانيين يتابعون هذه التطورات بقلق بالغ. وأشار إلى أن السؤال الذي يدور الآن هو ماذا سيكون رد فعل أوكرانيا والغرب وما هي الخطوة الروسية المقبلة. وحذر قائد الناتو في أوروبا يوم الأحد من أن القوات الروسية المنتشرة على الحدود الشرقية لأوكرانيا قادرة على شن عملية تمتد حتى مولدوفا. قمة الدول الصناعية الكبرى أوباما: العقوبات الغربية على موسكو ستؤثر على الاقتصاد الروسي. ويلقي ضم روسيا لمنطقة القرم بظلاله على قمة مجموعة السبع، التي كان مزمعا عقدها منذ فترة طويلة، بشأن تهديدات الأمن النووي. ومن المتوقع أن يبحث زعماء المجموعة موقفا موحدا حيال الأزمة. وأكد الرئيس الأمريكي على أن أوروبا والولايات المتحدة متفقون على دعم الحكومة الأوكرانية وشعبها، مشيرا إلى أن العقوبات التي فرضت على موسكو ستؤثر على الاقتصاد الروسي. ومن المقرر أن يلتقي وزير الخارجية الأمريكي، جون كيري، مع نظيره الروسي، سيرغي لافروف، على هامش قمة مجموعة السبع. انقطاع الكهرباء من جهة أخرى، شكا سكان محليون في بعض مناطق القرم من انقطاع الكهرباء في وقت متأخر من الأحد. ولف الظلام العديد من المدن من بينها بعض أحياء العاصمة سيمفربول. وقالت شركة توريد الكهرباء في القرم ""كريمنيرغو"" في بيان بث على موقعها الإلكتروني إن عطلا فنيا أصاب أحد الخطوط التي تديرها شركة الكهرباء الوطنية الأوكرانية ""اوكرينيرغو"". ولم يتسن الحصول على تعليق من اوكرينيرغو، ولم يصدر أيضا تأكيد مستقل حول سبب انقطاع الكهرباء. ضم القرم وضمت روسيا القرم إليها عقب استفتاء أجري في المنطقة في 16 مارس/آذار. وجاءت الخطوة الروسية بعد أن أطاحت احتجاجات بالرئيس الأوكراني السابق الموالي لروسيا فيكتور يانوكوفيتش. وأكدت روسيا أنها تحركت لحماية مواطني القرم المتحدرين من أصول روسية ضد من وصفتهم ""بالفاشيين"" الذين انتقلوا إليها من البلد الأم أوكرانيا. وردت الولايات المتحدة والاتحاد الأوروبي بفرض سلسلة من العقوبات ضد أفراد من بينهم مسؤولون بارزون اتهمتهم واشنطن وبروكسل بلعب دور في ضم القرم. موالون لروسيا يتظاهرون في مدينة مدينة دونيتسك، شرقي أوكرانيا. تسيطر قوات روسية حاليا على معظم القواعد العسكرية الأوكرانية في القرم.
"""
summary = generate_summary(document_text, summary_length=3)
print(summary)

قلق بالغ وقال مارك لوين، مراسل بي بي سي في القرم، إن القوات الروسية تسيطر بشكل كامل على القاعدة، ونقلت الجنود الأوكرانيين بعيدا إلى مكان مجهول. وأكدت روسيا أنها تحركت لحماية مواطني القرم المتحدرين من أصول روسية ضد من وصفتهم ""بالفاشيين"" الذين انتقلوا إليها من البلد الأم أوكرانيا. وتسيطر قوات روسية حاليا على معظم القواعد العسكرية الأوكرانية في القرم التي أعلنت موسكو ضمها للاتحاد الروسي بعد استفتاء أجرته السلطات المحلية هناك.


In [9]:


# Load BBC dataset
df = pd.read_csv('bbc_news_with_articles_and_extractive_summary.csv')

print(df.head())

   Unnamed: 0  Title                                            Article  \
0           0    289  Musicians to tackle US red tape\n\nMusicians' ...   
1           1    262  U2's desire to be number one\n\nU2, who have w...   
2           2    276  Rocker Doherty in on-stage fight\n\nRock singe...   
3           3     60  Snicket tops US box office chart\n\nThe film a...   
4           4     74  Ocean's Twelve raids box office\n\nOcean's Twe...   

                                             Summary       Category  \
0  Nigel McCune from the Musicians' Union said Br...  entertainment   
1  But they still want more.They have to want to ...  entertainment   
2  Babyshambles, which he formed after his acrimo...  entertainment   
3  A Series of Unfortunate Events also stars Scot...  entertainment   
4  Ocean's Twelve, the crime caper sequel starrin...  entertainment   

                                  extractive_summary  
0  Nigel McCune from the Musicians' Union said Br...  
1  They have

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))

def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenization
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove stopwords and punctuation
    return tokens



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jameelamer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jameelamer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Apply preprocessing
df["processed_Article"] = df["Article"].apply(preprocess)
df["processed_summary"] = df["Summary"].apply(preprocess)


In [12]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Create Dictionary and Corpus
dictionary = Dictionary(df["processed_Article"])
corpus = [dictionary.doc2bow(text) for text in df["processed_Article"]]

# Train LDA model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Print topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)


(0, '0.017*"said" + 0.007*"people" + 0.007*"software" + 0.005*"search" + 0.005*"would"')
(1, '0.023*"said" + 0.013*"mr" + 0.008*"would" + 0.007*"people" + 0.006*"government"')
(2, '0.010*"game" + 0.007*"said" + 0.005*"like" + 0.005*"time" + 0.004*"new"')
(3, '0.013*"said" + 0.009*"mobile" + 0.009*"people" + 0.007*"technology" + 0.007*"music"')
(4, '0.010*"said" + 0.006*"best" + 0.005*"year" + 0.005*"film" + 0.005*"also"')


In [13]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model
#bert_model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_summary(text, lda_model, dictionary, bert_model, num_sentences=3):
    # Split into sentences
    sentences = nltk.sent_tokenize(text)
    if len(sentences) <= num_sentences:
        return text  # If too short, return the full text

    # Compute sentence embeddings
    sentence_embeddings = bert_model.encode(sentences)

    # Compute topic distribution
    bow_vector = dictionary.doc2bow(preprocess(text))
    topic_distribution = lda_model.get_document_topics(bow_vector)

    # Get dominant topic
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]

    # Get topic words
    topic_words = [word for word, _ in lda_model.show_topic(dominant_topic)]

    # Compute similarity to topic words
    topic_embedding = bert_model.encode(" ".join(topic_words))
    sentence_scores = [cosine_similarity([sent_emb], [topic_embedding])[0][0] for sent_emb in sentence_embeddings]

    # Select top sentences
    summary_sentences = [sentences[i] for i in np.argsort(sentence_scores)[-num_sentences:]]
    return " ".join(summary_sentences)

# Generate summaries
df["generated_summary"] = df["Article"].apply(lambda x: extract_summary(x, lda_model, dictionary, bert_model))


In [14]:
from rouge import Rouge

rouge = Rouge()

def compute_rouge(reference, generated):
    scores = rouge.get_scores(generated, reference)
    return scores[0]  # Returns ROUGE-1, ROUGE-2, ROUGE-L scores

# Evaluate summaries
df["rouge_scores"] = df.apply(lambda row: compute_rouge(row["Summary"], row["generated_summary"]), axis=1)

# Display average ROUGE scores
rouge_l_scores = [score["rouge-l"]["f"] for score in df["rouge_scores"]]
print(f"Average ROUGE-L Score: {sum(rouge_l_scores) / len(rouge_l_scores):.4f}")


Average ROUGE-L Score: 0.3871


In [17]:
# Display average ROUGE scores
rouge_l_scores = [score["rouge-2"]["f"] for score in df["rouge_scores"]]
print(f"Average ROUGE-2 Score: {sum(rouge_l_scores) / len(rouge_l_scores):.4f}")

Average ROUGE-2 Score: 0.2725


In [18]:
# Display average ROUGE scores
rouge_l_scores = [score["rouge-1"]["f"] for score in df["rouge_scores"]]
print(f"Average ROUGE-1 Score: {sum(rouge_l_scores) / len(rouge_l_scores):.4f}")

Average ROUGE-1 Score: 0.3988


In [15]:
def compute_similarity(reference, generated):
    ref_embedding = bert_model.encode(reference)
    gen_embedding = bert_model.encode(generated)
    return cosine_similarity([ref_embedding], [gen_embedding])[0][0]

df["cosine_similarity"] = df.apply(lambda row: compute_similarity(row["Summary"], row["generated_summary"]), axis=1)
print(f"Average Cosine Similarity: {df['cosine_similarity'].mean():.4f}")


Average Cosine Similarity: 0.6558


In [13]:
from gensim.models import CoherenceModel

coherence_model = CoherenceModel(model=lda_model, texts=df["processed_Article"], dictionary=dictionary, coherence="c_v")
coherence_score = coherence_model.get_coherence()
print(f"Topic Coherence Score: {coherence_score:.4f}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Topic Coherence Score: 0.3597


In [15]:
from sklearn.model_selection import train_test_split

# Load dataset 
BBC_df = pd.read_csv("bbc_news_summary_with_articles.csv")
BBC_df.dropna(subset=['Article', 'Summary'], inplace=True)
# Split into 80% train and 20% test
train_df, test_df = train_test_split(BBC_df, test_size=0.2, random_state=42)

# Show dataset sizes
print(f"Train Size: {len(train_df)}, Test Size: {len(test_df)}")


Train Size: 1780, Test Size: 445


In [16]:
from sentence_transformers import InputExample

# Create InputExamples for training
train_examples = []
for i, row in train_df.iterrows():
    train_examples.append(InputExample(
        texts=[row['Article'], row['Summary']],
        label=1.0  # Label of 1.0 for positive (similar) pairs
    ))



In [17]:
print(train_df.columns)

Index(['Title', 'Article', 'Summary', 'Category'], dtype='object')


In [5]:
from sentence_transformers import SentenceTransformer, losses, InputExample,models,util
from torch.utils.data import DataLoader
# Convert the examples into a DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

bert_model = SentenceTransformer('all-MiniLM-L6-v2')
# Use CosineSimilarityLoss for fine-tuning the model
train_loss = losses.CosineSimilarityLoss(bert_model)

In [6]:
# Train the model
bert_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=4,  # Set this to more if needed
    warmup_steps=100,
    output_path='./sbert_finetuned/sbert_bbc_finetuned'
)

Step,Training Loss


In [18]:
from transformers import AutoTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
# Apply preprocessing
test_df["processed_Article"] = test_df["Article"].apply(preprocess)
test_df["processed_summary"] = test_df["Summary"].apply(preprocess)

In [19]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Create Dictionary and Corpus
dictionary = Dictionary(test_df["processed_Article"])
corpus = [dictionary.doc2bow(text) for text in test_df["processed_Article"]]

# Train LDA model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Print topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.020*"said" + 0.010*"mr" + 0.007*"would" + 0.005*"labour" + 0.004*"government"')
(1, '0.009*"said" + 0.007*"best" + 0.006*"music" + 0.005*"one" + 0.005*"years"')
(2, '0.012*"said" + 0.008*"would" + 0.006*"mr" + 0.005*"new" + 0.004*"also"')
(3, '0.016*"said" + 0.004*"us" + 0.004*"also" + 0.003*"would" + 0.003*"mr"')
(4, '0.009*"said" + 0.006*"us" + 0.004*"film" + 0.004*"software" + 0.004*"new"')


In [20]:
# Generate summaries
test_df["generated_summary"] = test_df["Article"].apply(lambda x: extract_summary(x, lda_model, dictionary, bert_model))


In [21]:

# Evaluate summaries
test_df["rouge_scores"] = test_df.apply(lambda row: compute_rouge(row["Summary"], row["generated_summary"]), axis=1)

# Display average ROUGE scores
rouge_l_scores = [score["rouge-l"]["f"] for score in df["rouge_scores"]]
print(f"Average ROUGE-L Score after finetune: {sum(rouge_l_scores) / len(rouge_l_scores):.4f}")

Average ROUGE-L Score after finetune: 0.3630


In [22]:
test_df["cosine_similarity"] = test_df.apply(lambda row: compute_similarity(row["Summary"], row["generated_summary"]), axis=1)
print(f"Average Cosine Similarity: {df['cosine_similarity'].mean():.4f}")

Average Cosine Similarity: 0.6261
